# Conversational Threads

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Group traces into threads


In [3]:
import uuid
thread_id = uuid.uuid4()

In [4]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [8]:
question = "Who was Subhash Chandra Bose and what did he do?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

Subhash Chandra Bose was an Indian nationalist leader who played a significant role in the Indian independence movement against British rule. He is best known for his leadership of the Indian National Army (INA) and his efforts to seek international support for India's independence. Bose advocated for complete independence and was a prominent figure in the struggle for freedom, emphasizing the need for armed resistance against colonial rule.


In [9]:
question = "Was the Indian National Army involved in the Indian freedom struggle?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

Yes, the Indian National Army (INA) was involved in the Indian freedom struggle. It was formed to fight against British rule in India during World War II, aiming to secure independence through armed struggle. The INA played a significant role in inspiring nationalist sentiments among Indians.


In [10]:
question = "Who were the founders of the Indian National Army?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

The Indian National Army (INA) was founded by Subhas Chandra Bose. Other key figures involved in its formation included Rash Behari Bose and Mohan Singh. The INA aimed to secure Indian independence from British rule.
